# StockSense - Model Training
## Training LSTM model for stock prediction

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from src.data_loader import StockDataLoader
from src.features import FeatureEngineering
from src.train import ModelTrainer
from src.predict import Predictor
import config

%matplotlib inline

## 1. Load and Prepare Data

In [ ]:
ticker = 'AAPL'

# Load data
loader = StockDataLoader(ticker)
df = loader.fetch_data()

# Create features
fe = FeatureEngineering(df)
df_processed = fe.create_all_features()

print(f"Dataset shape: {df_processed.shape}")

## 2. Prepare Sequences

In [ ]:
trainer = ModelTrainer(df_processed)
X, y = trainer.prepare_sequences(seq_length=config.SEQUENCE_LENGTH)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

## 3. Split Data

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = trainer.split_data(X, y)

## 4. Train Model

In [ ]:
history = trainer.train(X_train, y_train, X_val, y_val, ticker)
trainer.save_scaler(ticker)

## 5. Training History

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(history.history['accuracy'], label='Train')
ax1.plot(history.history['val_accuracy'], label='Validation')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(history.history['loss'], label='Train')
ax2.plot(history.history['val_loss'], label='Validation')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 6. Evaluate Model

In [ ]:
predictor = Predictor(trainer.model, trainer.scaler, config.FEATURE_COLUMNS)
y_pred, predictions = predictor.batch_predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, 
                           target_names=['Down', 'Sideways', 'Up']))

## 7. Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Down', 'Sideways', 'Up'],
           yticklabels=['Down', 'Sideways', 'Up'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## 8. Sample Predictions

In [ ]:
result = predictor.predict(df_processed)

print(f"\n🔮 Latest Prediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"\nProbabilities:")
print(f"  Down: {result['probabilities']['down']:.2%}")
print(f"  Sideways: {result['probabilities']['sideways']:.2%}")
print(f"  Up: {result['probabilities']['up']:.2%}")